# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, we might want to include an image classifier in a smart phone app. To do this, we'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, we have trined an image classifier to recognize different species of flowers, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, it can be seen a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

In [1]:
# Imports here
import matplotlib.pyplot as plt
import numpy as np
import json
import torch
from torch import nn 
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets,models
from PIL import Image
from network import Network
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Loading the data

Here we use `torchvision` to load the data ([documentation](http://pytorch.org/docs/master/torchvision/transforms.html#)). The data should be included alongside this notebook, otherwise, [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, we'll apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

For all three sets we have normalized the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. This converts the values of each color channel to be between -1 and 1 instead of 0 and 1.

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
train_transforms = transforms.Compose([transforms.Resize(256),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


train_dataset = datasets.ImageFolder(train_dir, transform = train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform = test_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform = test_transforms)


train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 64)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size = 64)

### Label mapping

We'll also need to load in a mapping from category label to category name. We can find this in the file `cat_to_name.json`. It's a JSON object which can be read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give us a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, we should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

In [ ]:
model = models.densenet201(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [ ]:
input_size = 1920
hidden_sizes = [512]
output_size = 102
dropout = 0.5
classifier = Network(input_size,hidden_sizes,output_size,dropout)
model.classifier = classifier


In [ ]:
criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
cuda = torch.cuda.is_available()

In [ ]:
def validate(model, loader):
    test_loss = 0
    accuracy = 0
    if cuda:
        model.cuda()
    for images, labels in valid_loader:
        if cuda:
            images,labels = images.cuda(), labels.cuda()
        
        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        top_p,top_class = ps.topk(1,dim=1)
        equality = (top_class == labels.view(*top_class.shape))
        accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
    
    return test_loss, (accuracy*100)

In [ ]:
epochs = 40
print_every = 25
steps = 0

if cuda:
    model.cuda()

for e in range(epochs):
    running_loss = 0
    model.train()
    
    for inputs, labels in train_loader:
        steps += 1
        optimizer.zero_grad()
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
        
            with torch.no_grad():
                validation_loss, accuracy = validate(model, valid_loader)
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(validation_loss/len(valid_loader)),
                  "Validation Accuracy: {:.2f} %".format(accuracy/len(valid_loader)))
            
            running_loss = 0
            model.train()

Epoch: 1/40..  Training Loss: 4.409..  Validation Loss: 4.085..  Validation Accuracy: 14.76 %
Epoch: 1/40..  Training Loss: 3.938..  Validation Loss: 3.371..  Validation Accuracy: 34.76 %
Epoch: 1/40..  Training Loss: 3.316..  Validation Loss: 2.613..  Validation Accuracy: 45.66 %
Epoch: 1/40..  Training Loss: 2.704..  Validation Loss: 2.061..  Validation Accuracy: 60.37 %
Epoch: 2/40..  Training Loss: 2.044..  Validation Loss: 1.709..  Validation Accuracy: 69.36 %
Epoch: 2/40..  Training Loss: 2.036..  Validation Loss: 1.350..  Validation Accuracy: 76.14 %
Epoch: 2/40..  Training Loss: 1.797..  Validation Loss: 1.218..  Validation Accuracy: 76.27 %
Epoch: 2/40..  Training Loss: 1.652..  Validation Loss: 1.014..  Validation Accuracy: 81.69 %
Epoch: 3/40..  Training Loss: 1.171..  Validation Loss: 0.928..  Validation Accuracy: 81.07 %
Epoch: 3/40..  Training Loss: 1.395..  Validation Loss: 0.807..  Validation Accuracy: 83.54 %
Epoch: 3/40..  Training Loss: 1.341..  Validation Loss: 0.75

## Testing our network (Validation)

It's good practice to test our trained network on test data, images the network has never seen either in training or validation. This will give ous a good estimate for the model's performance on completely new images. We run the test images through the network and measure the accuracy, the same way we did the validation.

In [ ]:
model.eval()
with torch.no_grad():
    test_loss, accuracy = validate(model, test_loader)

print("Test Accuracy: {:.2f} %".format(accuracy/len(test_loader)))


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. We want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. We can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

We will want to completely rebuild the model later so you can use it for inference.

In [ ]:
model.class_to_idx = train_dataset.class_to_idx

checkpoint = {
    'batch_size': 64,
    'input_size' : input_size,
    'hidden_sizes': hidden_sizes,
    'cnn_architecture' : 'densenet201',
    'lr' : 0.001,
    'classifier' : classifier,
    'epochs': epochs,
    'optimizer': optimizer.state_dict(),
    'state_dict': model.state_dict(),
    'class_to_idx': model.class_to_idx    
}
torch.save(checkpoint, './checkpoints/densenet201_epoch_50_dropout_05_lr_0_001.pth')

## Loading the checkpoint

A function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    learning_rate = checkpoint['lr']
    model = getattr(torchvision.models, checkpoint['cnn_architecture'])(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.epochs = checkpoint['epochs']
    model.optimizer = checkpoint['optimizer']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model
load_checkpoint('./checkpoints/densenet201_epoch_50_dropout_05_lr_0_001.pth')

## Testing our network (Training)

In [ ]:
!git clone https://github.com/GabrielePicco/deep-learning-flower-identifier
!pip install requests
!pip install airtable
import sys
sys.path.insert(0, 'deep-learning-flower-identifier')
from test_model_pytorch_facebook_challenge import calc_accuracy

calc_accuracy(load_checkpoint('./checkpoints/densenet201_epoch_50_dropout_05_lr_0_001.pth'), input_image_size=224)

# Inference for classification

A function to use a trained network for inference. That is, we pass an image into the network and predict the class of the flower in the image. The function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

## Image Preprocessing

First, we resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then we'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. We need to convert the values. It's easiest with a Numpy array, which we can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. We thus, subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. We can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:

def process_image(image_path):  
    image = Image.open(image_path)
    size = 255
    width, height = image.size
    if height > width:
        height = int(max(height * 255 / width, 1))
        width = int(255)
    else:
        width = int(max(width * 255 / height, 1))
        height = int(255)            
    resized_image = image.resize((width, height))
    
    size = 224
    width, height = resized_image.size
    x0 = (width - 224) / 2
    y0 = (height - 224) / 2
    x1 = x0 + 224
    y1 = y0 + 224
    cropped_image = resized_image.crop((x0, y0, x1, y1))

    image = np.array(cropped_image) / 255.
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])     
    image = (image - mean) / std
        
    image = image.transpose((2, 0, 1))
    
    return image

To check our work, the function below converts a PyTorch tensor and displays it in the notebook. If our `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    
    if ax is None:
        fig, ax = plt.subplots()
    image = image.transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. We need to convert from these indices to the actual class labels using `class_to_idx`. We also invert the dictionary so we can get a mapping from index to class as well.

This method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    model.cpu()
    model.eval()
    
    image = process_image(image_path)
    image = torch.from_numpy(image)
    image = image.type(torch.FloatTensor).unsqueeze(0)
    
    with torch.no_grad():
        output = model.forward(image)
        ps = torch.exp(output)
        
        top_p, top_class = ps.topk(topk)
        probable_classes = []
        
        for label in top_class.numpy()[0]:
            probable_classes.append(list(model.class_to_idx.keys())[list(model.class_to_idx.values()).index(label)])
    return top_p.numpy()[0], probable_classes

## Sanity Checking

Now that you can use a trained model for predictions, we can check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Using `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. 

In [ ]:
image_files = {train_dir + '/18/image_04313.jpg' : '#F87B7B', 
               valid_dir + '/13/image_05759.jpg' : '#F69A73',
               test_dir + '/41/image_02302.jpg' : '#F77CD4'
              }

for image_file,color in image_files.items():

    top_p, probable_classes = predict(image_file, model)

    fig = plt.figure(figsize=(7,7))
    img = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
    bgnd = plt.subplot2grid((15,9), (9,2), colspan=5, rowspan=5)

    image = Image.open(image_file)
    img.axis('off')
    img.set_title(f'{cat_to_name[probable_classes[0]]}')
    img.imshow(image)

    labels = []
    for classes in probable_classes:
        labels.append(cat_to_name[classes])

    bgnd.set_yticks(np.arange(5))
    bgnd.set_yticklabels(labels)
    bgnd.set_xlabel('Probability')
    bgnd.invert_yaxis()
    bgnd.barh(np.arange(5), top_p, xerr=0, align='center', color=color)

    plt.show()
